In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from flask import Flask, render_template, jsonify
from flask_sqlalchemy import SQLAlchemy 
from datetime import datetime
import time

In [2]:
class BlinkParser():
    CHROME_PATH = "/Users/ik/bin/chromedriver"
    DIRECTORY_URL = 'https://locations.blinkfitness.com/index.html'
    
    branch_status_dict = {
        0: ['Open Now - Closes at'],
        1: ['Opening Soon'], 
        2: ['Temporarily Closed', 'Closed'],
        3: ['Closed - Opens at']}

    def __init__(self):
        self.driver = None
        self.branch_directory_urls = []
        self.branch_info = []
        
    def load_chromedriver(self, path):
        chrome_options = Options()
        chrome_options.page_load_strategy = 'eager'
        return webdriver.Chrome(path, options=chrome_options)
    
    def parse(self):
        self.driver = self.load_chromedriver(BlinkParser.CHROME_PATH)
        self.driver.get(BlinkParser.DIRECTORY_URL)
        wait = WebDriverWait(self.driver, 10)
    
        branch_links = wait.until(lambda d: d.find_elements_by_tag_name('a'))
        
        # not including virginia beach since VA does not have standard directory like all other states
        self.branch_directory_urls = self.find_hrefs(
            list_a_tags=branch_links,
            url_starts_with='https://locations.blinkfitness.com/',
            url_does_not_include=['index','search','virginia-beach'])
        
        # if urls not parsed yet, then sleep and retry after 1 second
        while len(self.branch_directory_urls) == 0:
            time.sleep(1)
            
        self.parse_branch_info()
        
        self.driver.quit()
  

    def find_hrefs(self, list_a_tags, url_starts_with, url_does_not_include):
        list_urls = []

        for link in list_a_tags:
            url_string = link.get_attribute('href')

            if url_string.startswith(url_starts_with):
                if not any(x in url_string for x in url_does_not_include):
                    list_urls.append(url_string)
        return list_urls        
    
    def parse_branch_info(self):
        for url in self.branch_directory_urls:
            self.driver.get(url)
    
            # parse elements containing individual branch information
            wait = WebDriverWait(self.driver, 10)
            cities = wait.until(lambda d: d.find_elements_by_class_name('Directory-cityContainer'))
            
            for city in cities:
                branches = city.find_elements_by_class_name('Directory-listTeaser')
                
                for branch in branches:
                    temp_branch = {
                        'state': url[-2:].upper(),
                        'city': city.find_element_by_class_name('Directory-cityName').text,
                        'street': branch.find_element_by_class_name('Teaser-address').text,
                        'title': branch.find_element_by_class_name('Teaser-title').text,
                        'phone': branch.find_element_by_class_name('Teaser-phone').text,
                        'url': branch.find_element_by_class_name('Teaser-titleLink').get_attribute('href')}
                    
                    self.branch_info.append(temp_branch)

    def get_urls(self):
        return [branch['url'] for _,branch in enumerate(self.branch_info)]
    
    @staticmethod
    def status_to_code(status):    
        for status_code, status_text_list in BlinkParser.branch_status_dict.items():
            if status in status_text_list:
                return status_code
        return None
     
    def get_status_code(self, url):
        self.driver.get(url)
        wait = WebDriverWait(self.driver, 3)
        
        try:
            status = wait.until(lambda d: d.find_element_by_class_name('Hours-statusText')).text 
        except:
            # for branches that have no current status (usually branches that have not been opened yet)
            status = wait.until(lambda d: d.find_element_by_class_name('Core-openingDate')).text
    
        return BlinkParser.status_to_code(status)
        
    def parse_capacity(self):
        # load new driver in case connection refused from too many requests from initial parse
        self.driver = self.load_chromedriver(BlinkParser.CHROME_PATH)
        
        urls = self.get_urls()
        capacities = []
        
        for url in urls:
            status_code = self.get_status_code(url)
            
            # status code corresponds to dictionary at beginning of class (0 = branch is open)
            # find_elements used with walrus operator to avoid error for certain webpages where Core-capacityStatus shows up for unopened branches (it shouldn't)
            if not status_code and len(cap_element := self.driver.find_elements_by_class_name('Core-capacityStatus')) > 0:
                capacity = cap_element[0].text
            else:
                capacity = None
            
            capacities.append({
                'title': self.driver.find_element_by_class_name('LocationName-geo').text,
                'timestamp': datetime.now(),
                'status_code': status_code,
                'capacity': capacity 
            })
            
        self.driver.quit()
        return capacities

In [3]:
app = Flask(__name__)

app.config['ENV'] = 'development'
#app.config['SECRET_KEY'] = ''

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blink.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# database setup
db = SQLAlchemy(app)

@app.route('/')
def home():
    return render_template("base.html")


class Branch(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(30), unique=True, nullable=False)
    phone = db.Column(db.String(12), unique=True, nullable=False)
    url = db.Column(db.String(100), nullable=False)
    address = db.relationship('Address', backref='branch', lazy=True, uselist=False)
    
    def __repr__(self):
        return f"Branch: {self.title.title()}"
    
class Address(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    branch_id = db.Column(db.Integer, db.ForeignKey('branch.id'), nullable=False)
    state = db.Column(db.String(2), nullable=False) # TODO: make this limited to state abbreviations
    city = db.Column(db.String(30), nullable=False)
    street = db.Column(db.String(100), unique=True, nullable=False)
    
    def __repr__(self):
        return f"Address: {self.street}, {self.city}, {self.state}"
    
class Capacity(db.Model):
    id = db.Column(db.Integer,primary_key=True)
    branch_id = db.Column(db.Integer, db.ForeignKey('branch.id'), nullable=False)
    capacity = db.Column(db.String(100))
    status_code = db.Column(db.Integer)
    timestamp = db.Column(db.DateTime)

In [4]:
# clear tables if starting from scratch 
def refresh_tables():
    db.drop_all()
    db.create_all()
    
# adds and commits python objects mapped using ORM to SQLAlchemy database
def add_and_commit_to_db(db_object):
    db.session.add(db_object)
    db.session.commit()

# store branch info dictionaries into db 
# parser.branch_info should have been populated prior using parser.parse() 
def main(parser):
    
    for branch in parser.branch_info:
        # Address data stored using separate db model (tied to Branch) for abstraction
        branch_address = Address(
            state = branch['state'],
            city = branch['city'],
            street = branch['street']
        )
        
        new_branch = Branch(
            title = branch['title'],
            address = branch_address,
            phone = branch['phone'],
            url = branch['url']
        )
        
        add_and_commit_to_db(new_branch)

In [5]:
# scrape capacity data from individual branch pages and store into database
def capacity(parser):
    capacities = parser.parse_capacity()
    
    for cap in capacities:   
        # get branch ID from matching branch titles in Branch table and capacity data
        blink_branch_id = Branch.query.filter(Branch.title == cap['title']).first().id
        
        # raise error if no valid branch in Branch table
        if not blink_branch_id:
            raise NameError('No valid blink branch id for capacity reading')
        
        # store capacity data as SQLAlchemy object
        new_capacity = Capacity(
            branch_id = blink_branch_id,
            status_code = cap['status_code'],
            capacity = cap['capacity'],
            timestamp = cap['timestamp']
        )
        
        add_and_commit_to_db(new_capacity)


### Initialize BlinkParser

In [6]:
parser = BlinkParser()

### Initial parsing setup
Only run this cell if you want to drop and recreate all tables (deleting old branch metadata), parse fresh data, and store into Branch / Address tables.

In [7]:
refresh_tables()
parser.parse()
main(parser)

### Parse capacity data (goal is to automate this portion)
Crawls each individual branch homepage for capacity (n/a, 25%, 50%, 75%, 100%) and stores into Capacity table.

In [35]:
capacity(parser)

### Print number of all capacity readings and all non-NA readings

In [36]:
all_readings = Capacity.query.all()
all_valid_readings = Capacity.query.filter(Capacity.capacity != None).all()

print('Number of capacity readings: ', len(all_readings))
print('Nnumber of non-NA capacity readings: ', len(all_valid_readings))

Number of capacity readings:  763
Nnumber of non-NA capacity readings:  672


### Query: join Capacity and Branch tables on branch_id filtering out readings with capacity n/a

In [20]:
caps = db.session.query(Capacity, Branch).join(Branch).filter(Capacity.capacity != None).all()

In [21]:
for cap in caps:
    print(f"{cap.Capacity.timestamp}  {cap.Branch.title} ({cap.Branch.address.state}): {cap_str[-8:-5] if (cap_str:=cap.Capacity.capacity) else None}")

2020-12-15 15:31:35.014090  Jacksonville (FL): 25%
2020-12-15 15:31:35.542752  Miramar (FL): 25%
2020-12-15 15:31:36.127613  Midway (IL): 50%
2020-12-15 15:31:36.875111  Merrionette Park (IL): 25%
2020-12-15 15:31:38.624974  Bridgeport (IL): 50%
2020-12-15 15:31:40.181774  Oak Lawn (IL): 50%
2020-12-15 15:31:40.818354  Evanston (IL): 50%
2020-12-15 15:31:41.561634  Beverly (MA): 25%
2020-12-15 15:31:42.293603  Medford (MA): 50%
2020-12-15 15:31:43.045792  Redford (MI): 25%
2020-12-15 15:31:43.636399  Warren (MI): 25%
2020-12-15 15:31:44.205334  Paramus (NJ): 75%
2020-12-15 15:31:44.803289  Lodi (NJ): 50%
2020-12-15 15:31:45.450101  Journal Square (NJ): 50%
2020-12-15 15:31:46.405215  Parsippany (NJ): 50%
2020-12-15 15:31:47.004894  Union (NJ): 50%
2020-12-15 15:31:47.656257  Clifton (NJ): 50%
2020-12-15 15:31:48.187261  Linden (NJ): 50%
2020-12-15 15:31:48.950594  South Orange (NJ): 75%
2020-12-15 15:31:50.836021  Irvington (NJ): 50%
2020-12-15 15:31:51.761267  Passaic (NJ): 50%
2020-1

In [26]:
branches = db.session.query(Branch, Address).join(Address).all()
len(branches)

109